<a href="https://www.kaggle.com/code/avikumart/healthcare-datasci-retinal-fundus-segmentation?scriptVersionId=266462890" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
from PIL import Image
import cv2
from scipy.ndimage import binary_opening
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-10-07 20:21:45.806110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759868506.038008      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759868506.105262      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/34_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/40_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/22_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/27_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/28_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/26_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/38_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/39_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/24_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/tr

In [2]:
# datasets path

# Dataset paths
train_images_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images"
train_manual_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/1st_manual"
train_mask_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/mask"
test_images_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images"
test_mask_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/mask"

def check_dir(path):
    if os.path.exists(path):
        print(f"{path} exists")
    else:
        print(f"{path} does not exist")
    return os.path.exists(path)

# check the dirs
check_dir(train_images_path)
check_dir(test_images_path)

/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images exists
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images exists


True

In [10]:
# load the images
def load_images(img_path):
    img = Image.open(img_path)
    if img.mode == "RGB":
        img = np.array(img)
        green_channel = img[:,:,1]
        return green_channel
    else:
        return np.array(img)

In [13]:
image = load_images("/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images/02_test.tif")
image.shape

(584, 565)

In [6]:
# pre-process the images
def preprocess_image(image):
    mean = np.mean(image)
    std = np.std(image) if np.std(image) != 0 else 1
    gcn_image = (image- mean)/std
    gcn_image = np.clip(gcn_image, 0, 255).astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(gcn_image)
    gamma = 1.2
    gamma_image = np.array(255 * (clahe_image / 255) ** (1.0 / gamma), dtype=np.uint8)
    return gamma_image

In [16]:
def extract_patches(image, mask, manual=None, patch_size=64):
    patches =  []
    labels = []
    height, width = image.shape
    step = patch_size // 4
    pad_size = patch_size // 2
    padded_image = np.pad(image, ((pad_size,pad_size),(pad_size, pad_size)), mode="constant")
    padded_mask = np.pad(mask, ((pad_size, pad_size), (pad_size, pad_size)), mode="constant")
    padded_manual = np.pad(manual, ((pad_size, pad_size), (pad_size, pad_size)), mode="constant")

    # loop over the image to extract the patches
    for i  in range(pad_size, height + pad_size, step):
        for j in range(pad_size, width + pad_size, step):
            if padded_mask[i,j] == 0:
                continue
            patch = padded_image[i - pad_size:i+pad_size, j - pad_size:j +pad_size]
            if patch.shape != (patch_size, patch_size):
                continue
            if manual is not None:
                label = padded_manual[i, j] // 255
                labels.append(label)
            patches.append(patch)


    patches = np.array(patches)
    labels = np.array(labels) if manual is not None else None
    if manual is not None:
        print(f"Extracted {len(patches)} patches: {np.sum(labels == 1)} vessel, {np.sum(labels == 0)} non-vessel")
    else:
        print(f"extracted {len(patches)} patches for prediction")
    return patches, labels

In [ ]:
# level-I balancing